In [1]:
import pyspark

In [2]:
print(f'pyspark: {pyspark.__version__}')

pyspark: 3.3.1


In [3]:
from pyspark import SparkContext 
sc = SparkContext.getOrCreate() 

# check that it really works by running a job
# example from http://spark.apache.org/docs/latest/rdd-programming-guide.html#parallelized-collections
data = range(10000) 
distData = sc.parallelize(data)
distData.filter(lambda x: not x&1).take(10)
# Out: [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [30]:
distFile = sc.textFile("data.txt")

# Saving and Loading SequenceFiles
rdd = sc.parallelize(range(1, 4)).map(lambda x: (x, "a" * x))
rdd.saveAsSequenceFile("path1/to/file")
sorted(sc.sequenceFile("path1/to/file").collect())

[(1, 'a'), (2, 'aa'), (3, 'aaa')]

In [4]:
sc._conf

In [5]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

''